In [ ]:
%matplotlib inline
from kidsdata import KissRawData, list_scan, get_scan

In [ ]:
# Temporary import for dev
import numpy as np
import matplotlib.pyplot as plt
from astropy.wcs import WCS

List available scans

In [ ]:
list_scan(source='Moon', obsmode="SKYRASTER", scan__gt=430)

Reading of the file header

In [ ]:
kd = KissRawData(get_scan(431))
kd

Basic Properties of the object

In [ ]:
kd.info()

In [ ]:
print(len(kd))

In [ ]:
kd.names # Available data in the file

In [ ]:
kd.header # Header from the binary file

In [ ]:
kd.kidpar # Basic kidpar present in the file

In [ ]:
## This would read everything... 
# kd.read_data()

In [ ]:
# Instead read only a subset : All common data + I & Q, for the first 10 kids of box B

list_data = " ".join(kd.names.DataSc +
                     kd.names.DataUc +
                     ["I", "Q"])

# kd.read_data(list_data=list_data, list_detector=kd.get_list_detector('B')[0:10], silent=True)
kd.read_data(list_data=list_data, list_detector=kd.get_list_detector('B'), silent=True)


Not formally needed but we can calibrate the data manualy here

In [ ]:
kd.calib_raw()

In [ ]:
fig = kd.plot_calib(2)
# fig.savefig('calib.pdf')

Compute the new diff coordinates, generate beammaps and plot them

In [ ]:
fig_beammap, (datas, wcs, popts) = kd.plot_beammap(coord='pdiff')

In [ ]:
# Manualy tweak the coordinates to center them
for key in ['x0', 'y0']:
    popts[key] -= np.nanmedian(popts[key])
kd._extended_kidpar = popts # save the geometry
kd.kidpar.loc[kd.list_detector]

In [ ]:
fig_geometry, fig_fwhm = kd.plot_kidpar() # plot the geometry

In [ ]:
# select good detector, ie within 60 arcmin of the center and fwhm 25 +- 10

sub_kidpar = kd.kidpar.loc[kd.list_detector]

pos = np.array([sub_kidpar["x0"], sub_kidpar["y0"]]) * 60  # arcmin
fwhm = np.array(np.abs(sub_kidpar['fwhm_x']) + np.abs(sub_kidpar['fwhm_y']))/2 * 60
ikid = np.where((np.sqrt(pos[0]**2 + pos[1]**2) < 60) & (np.abs(fwhm - 25) < 10))[0]

In [ ]:
data, weight, hits = kd.continuum_map(coord='pdiff', ikid=ikid, cdelt=0.02)
ax = plt.subplot(projection= WCS(data.header))
ax.imshow(data.data, origin='lower', vmin=-1000, vmax=15000)
ax.set_aspect("equal")
ax.set_xlim(10, 120)
ax.set_ylim(50, 140)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
kd.kidfreq[:,:,0:135] = 0
for _kidfreq, name in zip(kd.kidfreq[0:5], kd.list_detector[0:5]):
    plt.plot(_kidfreq[0], alpha=0.5, label=name)
plt.legend()

In [ ]:
flat = kd.kidfreq.reshape(kd.ndet, -1)
plt.imshow(np.corrcoef(flat[:, 0:1024*100]))
plt.colorbar()

In [ ]:
C_laser1_pos = kd.C_laser1_pos.reshape(-1, 1024)
fig, ax1 = plt.subplots()
for pos in C_laser1_pos[0:10]:
    _ = ax1.plot(pos)

A_masq = kd.A_masq.reshape(-1, 1024)
ax2 = ax1.twinx()
for mask in A_masq[0:10]: 
    ax2.plot(mask) 

In [ ]:
plt.imshow(C_laser1_pos-C_laser1_pos.mean(axis=0), aspect='auto')
plt.colorbar()